In [ ]:
import random
import math
import numpy as np

In [ ]:
def read_tsp_file(filename):
    node_coords = []
    with open(filename, 'r') as f:
        reading_coords = False
        for line in f:
            if "NODE_COORD_SECTION" in line:
                reading_coords = True
            elif "EOF" in line:
                break
            elif reading_coords:
                parts = line.strip().split()
                x, y = float(parts[1]), float(parts[2])
                node_coords.append((x, y))
    return node_coords

In [ ]:
def tour_length(tour, node_coords):
    total_distance = 0
    for i in range(len(tour) - 1):
        city_i = int(tour[i])
        city_j = int(tour[i + 1])
        total_distance += np.linalg.norm(np.array(node_coords[city_i]) - np.array(node_coords[city_j]))
    total_distance += np.linalg.norm(np.array(node_coords[int(tour[-1])]) - np.array(node_coords[int(tour[0])]))
    return total_distance

In [ ]:
def read_opt_tour_file(filename):
    opt_tour = []
    with open(filename, 'r') as f:
        reading_tour = False
        for line in f:
            if "TOUR_SECTION" in line:
                reading_tour = True
            elif "EOF" in line:
                break
            elif reading_tour:
                city_id = int(line.strip())
                if city_id != -1:
                    opt_tour.append(city_id - 1)  # Adjusting for 0-based indexing
    return opt_tour

In [ ]:
def generate_initial_solution(num_cities):
    return random.sample(range(num_cities), num_cities)

In [ ]:
def initialize_bats(num_bats, num_cities):
    bats = []
    for _ in range(num_bats):
        bat = {
            'position': generate_initial_solution(num_cities),
            'velocity': [0] * num_cities,
            'frequency': random.uniform(0, 1),
            'loudness': random.uniform(0, 1)
        }
        bats.append(bat)
    return bats

In [ ]:
def update_bat(bat, best_solution, alpha, gamma, num_cities):
    new_velocity = bat['velocity'] + (np.array(bat['position']) - np.array(best_solution)) * bat['frequency']
    new_position = np.clip(np.array(bat['position']) + new_velocity, 0, num_cities - 1)
    
    if random.random() < gamma:
        new_position = generate_initial_solution(num_cities)
    
    return new_position, new_velocity

In [ ]:
def bat_algorithm_tsp(node_coords, inner_iter, num_bats, alpha, gamma):
    num_cities = len(node_coords)
    best_solution = generate_initial_solution(num_cities)
    best_distance = tour_length(best_solution, node_coords)
    
    bats = initialize_bats(num_bats, num_cities)
    
    for _ in range(inner_iter):
        for bat in bats:
            new_position, new_velocity = update_bat(bat, best_solution, alpha, gamma, num_cities)
            
            new_distance = tour_length(new_position, node_coords)
            
            if new_distance < best_distance:
                best_solution = new_position
                best_distance = new_distance
        
        for bat in bats:
            bat['loudness'] *= alpha
            bat['frequency'] = 0.1 + 0.9 * random.random()
    
    return best_solution, best_distance

In [3]:
if __name__ == "__main__":
    tsp_filename = "berlin52.tsp"
    opt_tour_filename = "berlin52.opt.tour"
    
    node_coords = read_tsp_file(tsp_filename)
    opt_tour = read_opt_tour_file(opt_tour_filename)
    
    inner_iter = 100
    max_iter = 1000
    num_bats = 50
    alpha = 0.9
    gamma = 0.1
    
    best_distance = float('inf')

    for run in range(max_iter):
        best_solution, current_distance = bat_algorithm_tsp(node_coords, inner_iter, num_bats, alpha, gamma)
        
        if current_distance < best_distance:
            best_distance = current_distance

        print(f"Iteration {run + 1}'s best distance = {best_distance}")

    opt_distance = tour_length(opt_tour, node_coords)
    relative_error = abs(((opt_distance - best_distance) / opt_distance) * 100)

    print(f"Optimal distance: {opt_distance}")
    print(f"Best Distance of {num_bats} population Bat Algorithm: {best_distance}")
    print(f"Relative Percent error: {relative_error:.3f}%")


Iteration 1's best distance = 21822.94427270856
Iteration 2's best distance = 21822.94427270856
Iteration 3's best distance = 21822.94427270856
Iteration 4's best distance = 21126.641255557075
Iteration 5's best distance = 21126.641255557075
Iteration 6's best distance = 21126.641255557075
Iteration 7's best distance = 21126.641255557075
Iteration 8's best distance = 21126.641255557075
Iteration 9's best distance = 20211.950015748003
Iteration 10's best distance = 20211.950015748003
Iteration 11's best distance = 20211.950015748003
Iteration 12's best distance = 20211.950015748003
Iteration 13's best distance = 20211.950015748003
Iteration 14's best distance = 20211.950015748003
Iteration 15's best distance = 20211.950015748003
Iteration 16's best distance = 20211.950015748003
Iteration 17's best distance = 20211.950015748003
Iteration 18's best distance = 20211.950015748003
Iteration 19's best distance = 20125.020300264994
Iteration 20's best distance = 20125.020300264994
Iteration 21